###Delta table creation

In [0]:
from delta.tables import *

DeltaTable.create(spark) \
.tableName('employee_table')\
.addColumn('emp_id', 'INT')\
.addColumn('emp_name', 'STRING')\
.addColumn('gender', 'STRING')\
.addColumn('salary', 'INT')\
.addColumn('Dept', 'STRING')\
.property('description','table created for demo purpose')\
.location('dbfs:/FileStore/tables/delta/path_employee_demo')\
.execute()

In [0]:
%sql
select * from employee_table

emp_id,emp_name,gender,salary,Dept


###SQL Style Insert

In [0]:
%sql
insert into employee_table values(100, 'stephen','m',2000,'IT')

num_affected_rows,num_inserted_rows
1,1


In [0]:
display(spark.sql('select * from employee_table'))

emp_id,emp_name,gender,salary,Dept
100,stephen,m,2000,IT


In [0]:
#Dataframe Insert into method

In [0]:
from pyspark.sql.types import IntegerType, StringType
employee_table = [(300,'lara','f',6000,'Sales')]

employee_schema = StructType([\
    StructField('emp_id',IntegerType(), False),\
    StructField('emp_name',StringType(), True),\
    StructField('gender',StringType(), True),\
    StructField('salary',IntegerType(), True),\
    StructField('dept',StringType(), True),\
    ])

df1 = spark.createDataFrame(data=employee_table, schema = employee_schema)
display(df1)


emp_id,emp_name,gender,salary,dept
300,lara,f,6000,Sales


In [0]:
df1.write.insertInto('employee_table',overwrite=False)

###Insert using Temp View

In [0]:
df1.createOrReplaceTempView('delta_data')

In [0]:
%sql
select * from delta_data

emp_id,emp_name,gender,salary,dept
300,lara,f,6000,Sales


In [0]:
%sql
insert into employee_table
select * from delta_data

num_affected_rows,num_inserted_rows
1,1


In [0]:
###Spark SQL Insert
spark.sql('insert into employee_table select * from delta_data')

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql
select * from employee_table

emp_id,emp_name,gender,salary,Dept
300,lara,f,6000,Sales
300,lara,f,6000,Sales
300,lara,f,6000,Sales
100,stephen,m,2000,IT


###Method1 : SQL delete

In [0]:
%sql
delete from employee_table where emp_id = 100

In [0]:
#another method (pyspark delta table instance)
from delta.tables import *
from pyspark.sql.functions import *

deltaTable = DeltaTable.forName(spark, 'employee_demo')

#Declare the predicate by using a SQL - formatted string
deltaTable.delete('emp_id=300')

In [0]:
#multiple conditions using Sql predict
deltaTable.delete("emp_id = 500 and gender='f'")

In [0]:
#another method(Declare the predicate by using Spark SQL functions)
deltaTable.delete(col('emp_id')==600)